___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 5: Data analysis with NumPy and Pandas

## Lab work 5.2



#### Мета: 

* навчитися працювати із бібліотекою Pandas в Python.

### Завдання:

In [1]:
import pandas as pd
import numpy as np

Вивести версію та конфігурацію бібліотеки:

In [2]:
print(pd.__version__)
print(pd.describe_option())

1.0.5
compute.use_bottleneck : bool
    Use the bottleneck library to accelerate if it is installed,
    the default is True
    Valid values: False,True
    [default: True] [currently: True]
compute.use_numexpr : bool
    Use the numexpr library to accelerate computation if it is installed,
    the default is True
    Valid values: False,True
    [default: True] [currently: True]
display.chop_threshold : float or None
    if set to a float value, all float values smaller then the given threshold
    will be displayed as exactly 0 by repr and friends.
    [default: None] [currently: None]
display.colheader_justify : 'left'/'right'
    Controls the justification of column headers. used by DataFrameFormatter.
    [default: right] [currently: right]
display.column_space No description available.
    [default: 12] [currently: 12]
display.date_dayfirst : boolean
    When True, prints and parses dates with the day first, eg 20/01/2005
    [default: False] [currently: False]
display.date_year

In [3]:
df = pd.read_csv('Ecommerce Purchases.txt')

Скільки рядків і стовпців в наборі даних:

In [4]:
df.shape

(10000, 14)

Перевірити, чи в наборі даних містяться порожні значення:

In [5]:
print(df.isnull().sum().sum(), '- количество пустых значений')

0 - количество пустых значений


Яка середня ціна закупки (Purchase Price):

In [6]:
df['Purchase Price'].mean()

50.34730200000025

Скільки людей користуються англійською мовою "en" на веб-сайті:

In [7]:
df[df['Language'] == 'en']['Language'].count()

1098

Скільки людей має посаду «Lawyer»?

In [8]:
df[df['Job'].str.contains('Lawyer', case = False)].shape[0]

30

Скільки людей зробило покупку вранці та скільки людей зробило покупку після обіду?

In [9]:
am = df[df['AM or PM'] == 'AM']['AM or PM'].count()
pm = df['AM or PM'].count() - am
print(f'{am} людей совершили покупку утром,\n{pm} людей совершили покупку после обеда')

4932 людей совершили покупку утром,
5068 людей совершили покупку после обеда


Які 5 найпоширеніших назв вакансій?

In [10]:
df['Job'].value_counts()[:5]

Interior and spatial designer    31
Lawyer                           30
Social researcher                28
Purchasing manager               27
Designer, jewellery              27
Name: Job, dtype: int64

Хтось здійснив покупку, яка надійшла від Lot: "90 Wt", та якою була ціна придбання для цієї транзакції?

In [11]:
df[df['Lot'].str.contains('90 Wt', case = False)][['Lot', 'Company', 'Purchase Price']]

,Lot,Company,Purchase Price
513,90 WT,Bright PLC,75.1


Яка електронна адреса особи з таким номером кредитної картки: 4926535242672853 ?

In [12]:
df[df['Credit Card'] == 4926535242672853]['Email'].values[0]

'bondellen@williams-garza.com'

Скільки людей використовує American Express  і здійснили покупку на суму понад 99 доларів? ( >100 никто не покупал)

In [13]:
df[(df['CC Provider'] == 'American Express') & (df['Purchase Price'] > 99)]['CC Provider'].count()

8

Скільки людей мають кредитну картку, термін дії якої закінчується в 2025 році?

In [14]:
df[df['CC Exp Date'].str.contains('/25')]['CC Exp Date'].count()

1033

Які найкращі 5 найпопулярніших постачальників / хостів електронної пошти (наприклад, gmail.com, yahoo.com тощо ...).

In [15]:
def host(email):
    return email.split('@')[1]

df1 = df.copy()
df1['Email'] = df['Email'].map(host)
df1['Email'].value_counts()[:5]

hotmail.com     1638
yahoo.com       1616
gmail.com       1605
smith.com         42
williams.com      37
Name: Email, dtype: int64

Виведіть зведену таблицю по браузерах(Browser Info), посаді(Job), та кількості транзакцій :

In [16]:
df.groupby(['Browser Info', 'Job'])['Lot'].count().reset_index().sort_values('Lot')

,Browser Info,Job,Lot
0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_5_0 ...,"Psychotherapist, dance movement",1
6656,Mozilla/5.0 (compatible; MSIE 8.0; Windows NT ...,"Solicitor, Scotland",1
6657,Mozilla/5.0 (compatible; MSIE 8.0; Windows NT ...,Agricultural engineer,1
6658,Mozilla/5.0 (compatible; MSIE 8.0; Windows NT ...,Barrister's clerk,1
6659,Mozilla/5.0 (compatible; MSIE 8.0; Windows NT ...,Broadcast journalist,1
...,...,...,...
6675,Mozilla/5.0 (compatible; MSIE 8.0; Windows NT ...,"Research officer, political party",2
6368,Mozilla/5.0 (compatible; MSIE 7.0; Windows NT ...,Special educational needs teacher,2
5973,Mozilla/5.0 (compatible; MSIE 6.0; Windows NT ...,"Therapist, speech and language",2
6494,Mozilla/5.0 (compatible; MSIE 8.0; Windows 98;...,"Producer, radio",2


Створіть нову колонку "Actual price", яка утворюється із "Purchase Price" та націнки за принципом:
- якщо "Purchase Price" > 50, націнка 20%
- якщо "Purchase Price" > 100, націнка 10%
- в інших випадках націнка 30%
   

In [17]:
def new(old):
    if old > 50:
        new = old * 1.2
    elif old > 100:
        new = old * 1.1
    else:
        new = old * 1.3
    return new
        
df['Actual price'] = df['Purchase Price'].map(new)
df[['Purchase Price', 'Actual price']]

,Purchase Price,Actual price
0,98.14,117.768
1,70.73,84.876
2,0.95,1.235
3,78.04,93.648
4,77.82,93.384
...,...,...
9995,82.21,98.652
9996,25.63,33.319
9997,83.98,100.776
9998,38.84,50.492


Зробіть рангування набору даних по "Language" та "Actual price" в порядку спадання ціни.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rank.html

In [18]:
df2 = df[['Language', 'Actual price']].copy()
df2['Rank'] = df2.groupby('Language')['Actual price'].rank(ascending=False)
df2.sort_values('Actual price', ascending=False)

,Language,Actual price,Rank
2092,el,119.988,1.0
7807,de,119.988,1.0
9856,de,119.964,2.0
7604,el,119.940,2.0
3844,el,119.916,3.0
...,...,...,...
2783,de,0.026,1155.0
8656,ru,0.026,1155.0
6863,es,0.013,1095.0
2876,it,0.000,1086.0


Колонку "Language" (категоріальна змінна) "закодуйте", тобто утворити індикаторні колоник. В наборі не повинна залишитися колонка "Language".

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rank.html

In [19]:
cats = df['Language'].astype('category')
df['Language category'] = cats.cat.codes
df.drop(columns = 'Language', inplace = True)

d = dict(enumerate(cats.cat.categories))
print('Наша кодировка для Language:')
print(d)
df.head()

Наша кодировка для Language:
{0: 'de', 1: 'el', 2: 'en', 3: 'es', 4: 'fr', 5: 'it', 6: 'pt', 7: 'ru', 8: 'zh'}


,Address,Lot,AM or PM,Browser Info,Company,Credit Card,CC Exp Date,CC Security Code,CC Provider,Email,Job,IP Address,Purchase Price,Actual price,Language category
0,"16629 Pace Camp Apt. 448\nAlexisborough, NE 77...",46 in,PM,Opera/9.56.(X11; Linux x86_64; sl-SI) Presto/2...,Martinez-Herman,6011929061123406,02/20,900,JCB 16 digit,pdunlap@yahoo.com,"Scientist, product/process development",149.146.147.205,98.14,117.768,1
1,"9374 Jasmine Spurs Suite 508\nSouth John, TN 8...",28 rn,PM,Opera/8.93.(Windows 98; Win 9x 4.90; en-US) Pr...,"Fletcher, Richards and Whitaker",3337758169645356,11/18,561,Mastercard,anthony41@reed.com,Drilling engineer,15.160.41.51,70.73,84.876,4
2,Unit 0065 Box 5052\nDPO AP 27450,94 vE,PM,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,"Simpson, Williams and Pham",675957666125,08/19,699,JCB 16 digit,amymiller@morales-harrison.com,Customer service manager,132.207.160.22,0.95,1.235,0
3,"7780 Julia Fords\nNew Stacy, WA 45798",36 vm,PM,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0 ...,"Williams, Marshall and Buchanan",6011578504430710,02/24,384,Discover,brent16@olson-robinson.info,Drilling engineer,30.250.74.19,78.04,93.648,3
4,"23012 Munoz Drive Suite 337\nNew Cynthia, TX 5...",20 IE,AM,Opera/9.58.(X11; Linux x86_64; it-IT) Presto/2...,"Brown, Watson and Andrews",6011456623207998,10/25,678,Diners Club / Carte Blanche,christopherwright@gmail.com,Fine artist,24.140.33.94,77.82,93.384,3


Кінцевий набір даних збережіть у файл з розширенням csv.

In [20]:
df.to_csv('Ecommerce Purchases.csv')